In [1]:
import numpy as np
from random import shuffle, randint, uniform
from typing import List, Optional
"""
%pip install gdown
%gdown 1iOdA7wG3kKk3OgPDSC4Z4014eCoCKV6C 
code to download the sudoku.csv file from google drive.
Code snippet to extract sudokus from file 'sudoku.csv'
https://www.kaggle.com/datasets/bryanpark/sudoku
"""

quizzes = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)
for i, line in enumerate(open('sudoku copy.csv', 'r').read().splitlines()[1:]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s
quizzes = quizzes.reshape((-1, 9, 9))
solutions = solutions.reshape((-1, 9, 9))
s = list([list(i) for i in quizzes[0]])


In [2]:
import sudoku
solver = sudoku.Sudoku(s)

In [3]:
sol, ftns, ftns_history = solver.solve(cross_over="random candidate")

# Genetic Algorithm

In [4]:
def transform(problem_grid: List[List]) -> List[List]:
    """
    Transforms the problem_grid passed, testing the sudoku has three main comparisons.
      Assuming we have an untransformed grid, we have:
        1. All rows must have each number from 1 to 9 with no repetition
        2. All columns must have each number from 1 to 9 wtih no repetition
        3. All 3x3 squares must have each number from 1 to 9 with no repetition
      These last conditions are assuming we have a 9x9 grid with 81 numbers in total. 

      To be able to easily check for rows, columns and squares, we transform the grid, to convert each 3x3 square to a 1x9 row. 
      These way in the untransformed version we check rows and columns
      In the transformed version we check rows, and now we have checked every condition.
    Parameters:
      - problem_grid (list): Sudoku grid (Transformed or Untransformed).
    Returns (list).
      - transformed grid .
          If problem_grid is the original sudoku it converts it
          If the problem_grid is the already transformed sudoku, it unrolls it
    """
    file_lines = problem_grid
    problem_grid = [[] for i in range(len(file_lines))] #Empty grid
    sqrt_n = 3 # this parameter could change if one wants to generalize for sudokus of other dimensions
    for j in range(len(file_lines)): 
        line_values = [(int(value) if value != 0 else None) for value in file_lines[j]]
        for i in range(len(line_values)):
            problem_grid[
                int(i / sqrt_n) +
                int(j / sqrt_n) * sqrt_n
                ].append(line_values[i])
        
    return problem_grid

In [5]:
def solve(problem_grid: List[List],
          population_size: Optional[int] = 10000,
          selection_rate: Optional[float] = 0.8,
          max_generations_count: Optional[int] = 100,
          mutation_rate: Optional[float] = 0.4,
          random_proportion: Optional[float] = 0.5,
          verbose: Optional[bool] = False,
          ):
    """
    Solves a Sudoku puzzle using a genetic algorithm.
    It receives a problem grid written in different lines (Not one line with 81 numbers) and in the empty spaces, a 0 should appear
    In the "reading sudoku" chapter there is an example of how the initial sudoku should be formatted.

    Parameters:
        - problem_grid (list): An 9x9 sudoku grid.
        - population_size (int): The initial population size.
        - selection_rate (int)
        - max_generations_count (int)
        - mutation_rate (int)
      Returns:
        - population[0] (list): Solved sudoku.
        - best_fitness (int): last best_fitness, when not solved in the max_generations it gets the last one.
        - fitness_history (list): For plotting progress of learning
    """
    
    problem_grid = transform(problem_grid)
    
    def empty_grid(elem_generator: List[List] | None = None) -> List[List]:
        """
        Returns an empty Sudoku grid.
        Parameters:
        - elem_generator (function) (optional=None): Is is used to generate initial values of the grid's elements.
              If it's not given, all grid's elements will be "None".
        """
        return [
            [
                (None if elem_generator is None else elem_generator(i, j))
                for j in range(len(problem_grid))
            ] for i in range(len(problem_grid))
        ]

    def deep_copy_grid(grid: List[List]) -> List[List]:
        """
        Returns a deep copy of the grid argument.
        The reason why there is an empty grid and the a deep copy of a grid is the way python works with lists.
        For example, sample function and normal assignation of none to a list could modify the original list where
        it was defined from.
        By deep copying an empty grid we make sure, the original grid is not modified.
        Parameters:
            - grid (list)
        """

        return empty_grid(lambda i, j: grid[i][j])

    problem_grid = deep_copy_grid(problem_grid)

    def fitness(grid: List[List]) -> int:
        """
        Calculates the fitness function for a grid, the fitness integer is the number of repeated elements,
        either in a row, column or square.
        Parameters:
            - grid (list)
        Returns (int): The value of the fitness function for the input grid.
        """
        grid = np.array(grid)
        grid_2 = transform(grid) # Transforming

        ################# COUNT HOW MANY REPEATED NUMBERS ############################
        row = np.sum(9 - np.array([len(np.unique(r)) for r in grid_2]))
        column = np.sum(9 - np.array([len(np.unique(c)) for c in np.transpose(grid_2)]))
        square = np.sum(9 - np.array([len(np.unique(r)) for r in grid]))

        # Verification of given values
        actual = sum(x is not None for x in np.array(problem_grid).flatten()) - np.sum(np.array(problem_grid).flatten() == np.array(grid).flatten())
        fitness = row + column + square + actual
        return fitness

    def generate_initial_population(pop_size: int) -> List[List]:
        """
        Generates an initial population of size "population_size".
        Returns (list): An array of candidate grids.
        """

        candidates = []

        for k in range(pop_size):
            candidate = empty_grid()
            for i in range(len(problem_grid)):
                shuffled_sub_grid = [n for n in range(1, len(problem_grid) + 1)]
                shuffle(shuffled_sub_grid)

                for j in range(len(problem_grid)):
                    if problem_grid[i][j] is not None:
                        candidate[i][j] = problem_grid[i][j]

                        shuffled_sub_grid.remove(problem_grid[i][j])

                for j in range(len(problem_grid)):
                    if candidate[i][j] is None:
                        candidate[i][j] = shuffled_sub_grid.pop()

            candidates.append(candidate)

        return candidates

    def selection(candidates):
        """
        Returns the best portion ("selection_rate") of candidates based on their fitness function values (lower ones).
        Parameters:
          - candidates (list)
        Returns:
          - ordered candidates (list)
          - fitness of the first candidate (int)
          - fitness of the last candidate (int)
        """

        index_fitness = []
        for i in range(len(candidates)):
            index_fitness.append(tuple([i, fitness(candidates[i])]))
        index_fitness.sort(key=lambda elem: elem[1])

        selected_part = index_fitness[0: int(len(index_fitness) * selection_rate)]
        indexes = [e[0] for e in selected_part]

        return [candidates[i] for i in indexes], selected_part[0][1], selected_part[-1][1]

    population = generate_initial_population(population_size)
    best_fitness = 100 # initializing value with an improbable value
    fitness_history = [] # for plotting
    
    for i in range(max_generations_count):
        prev_fitness = best_fitness
        population, best_fitness, worst_fitness = selection(population) # selecting the best (selection_rate*population_size) candidates.
        if best_fitness == 0:  # If fitness equal to 0, then the solution is found.
            print(f"Generation {i} -Best candidate's fitness {best_fitness} -Worst candidate's fitness {worst_fitness}")
            break
        
        actual_pop_size = len(population)

        new_population_size = int((population_size - actual_pop_size) * (1 - random_proportion))
        random_population_size = int((population_size - actual_pop_size) * random_proportion)

        new_population = []

        #Cross-over
        for j in range(new_population_size):
            # Selecting two random candidates
            w1 = randint(0, actual_pop_size - 1)
            w2 = randint(0, actual_pop_size - 1)
            c1 = population[w1].copy() # Copy to avoid manipulating original list
            c2 = population[w2].copy()

            #Random cross-over
            cross_point = randint(0, 8)
            temp = c2[cross_point]
            c1[cross_point] = c2[cross_point] # Here we exchange a random row(That it was originally a square) from the first grid with another row from the second grid
            c2[cross_point] = temp

            if fitness(c1) < fitness(c2): # comparing best fitness of the two changes
                new_population.append(c1) # only appending the best candidate of the cross-over
            else:
                new_population.append(c2)

        random_population = generate_initial_population(random_population_size)

        # Mutation
        for candidate in new_population:
            if uniform(0, 1) < mutation_rate:
                random_sub_grid = randint(0, 8)
                possible_swaps = []
                for grid_element_index in range(len(problem_grid)):
                    if problem_grid[random_sub_grid][grid_element_index] is None:
                        possible_swaps.append(grid_element_index)
                if len(possible_swaps) > 1:
                    shuffle(possible_swaps)
                    first_index = possible_swaps.pop()
                    second_index = possible_swaps.pop()
                    tmp = candidate[random_sub_grid][first_index]
                    candidate[random_sub_grid][first_index] = candidate[random_sub_grid][second_index]
                    candidate[random_sub_grid][second_index] = tmp

        new_population = np.append(new_population, random_population, 0)

        population = np.append(population, new_population, 0)
        
        fitness_history.append(best_fitness)
        if verbose:
             print('Generation {} -Best candidate\'s fitness {} -Worst candidate\'s fitness {}'.format(i, best_fitness, worst_fitness)) # This is used to see progress in console of the solving

        # Resetting if local minimum
        if len(fitness_history) > 60 and len(np.unique(fitness_history[-49:])) == 1:
          print(population[0])
          population = generate_initial_population(population_size) # Create a whole new random population if last 20 fitness has not changed


        
    population, best_fitness, worst_fitness = selection(population)
    return population[0], best_fitness, fitness_history

# Solving

In [6]:
solution, best_fitness, fit_history = solve(s, selection_rate=0.05, mutation_rate = 0.6, random_proportion=0.6, max_generations_count=200, verbose=False)

Generation 16 -Best candidate's fitness 0 -Worst candidate's fitness 2
